In [7]:
import os, sys
sys.path.append('../..')

In [10]:
from dotenv import load_dotenv
from ck_helper import print_attrs, ready_langchain
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [9]:
from langchain_teddynote import logging
logging.langsmith('CVAL')

LangSmith 추적을 시작합니다.
[프로젝트명]
CVAL


#### <b>일반사용</b>

In [11]:
model_name = 'gpt-4o-mini'
llm = ChatOpenAI(temperature=.1, model_name=model_name)
question = '내가 이걸 몇번째 하고 있는지 아니?'
response = llm.invoke(question)

In [12]:
print_attrs(response)

 [content]
 죄송하지만, 제가 당신의 이전 대화나 활동을 기억할 수는 없습니다. 하지만 도움이 필요하시면 언제든지 말씀해 주세요!
 [additional_kwargs]
 {'refusal': None}
 [response_metadata]
 {'token_usage': {'completion_tokens': 32, 'prompt_tokens': 18, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}
 [type]
 ai
 [name]
 None
 [id]
 run-4cfdd444-ab8f-47f0-8b9f-06ed33abfb00-0
 [example]
 False
 [tool_calls]
 []
 [invalid_tool_calls]
 []
 [usage_metadata]
 {'input_tokens': 18, 'output_tokens': 32, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [15]:
print(response.content)
print(response.response_metadata)

죄송하지만, 제가 당신의 이전 대화나 활동을 기억할 수는 없습니다. 하지만 도움이 필요하시면 언제든지 말씀해 주세요!
{'token_usage': {'completion_tokens': 32, 'prompt_tokens': 18, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None}


#### <b>스트리밍</b>

In [20]:
response_string = ''
answer = llm.stream(question)
for token in answer:
    print(token.content, end='', flush=True)
    response_string += token.content
    # print_attrs(token)
    # break
print()
print(len(response_string))

죄송하지만, 제가 당신의 이전 대화나 활동을 추적할 수는 없습니다. 하지만 도움이 필요하시면 언제든지 말씀해 주세요!
65


#### <b>프롬프트 캐싱</b>

In [21]:
from langchain_teddynote.messages import stream_response

In [24]:
some_prompt = '''
Lorem Ipsum is simply dummy text of the printing and typesetting industry. 
Lorem Ipsum has been the industry's standard dummy text ever since the 1500s,
when an unknown printer took a galley of type and scrambled it to make a type specimen book.
It has survived not only five centuries, but also the leap into electronic typesetting,
remaining essentially unchanged. It was popularised in the 1960s with the release of 
Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing 
software like Aldus PageMaker including versions of Lorem Ipsum.

#Question: {}
'''

In [23]:
from langchain.callbacks import get_openai_callback

In [25]:
with get_openai_callback() as cb:
    answer = llm.invoke(some_prompt.format('2문장으로 요약해봐'))
    chached_tokens = answer.response_metadata['token_usage']['prompt_tokens_details']['cached_tokens']
    print(answer.content)
    print(f'캐싱 된 토큰: {chached_tokens}')
    print(f'Prompt tokens: {cb.prompt_tokens}')

Lorem Ipsum는 1500년대부터 인쇄 및 조판 산업의 표준 더미 텍스트로 사용되어 왔으며, 전자 조판으로의 전환에도 불구하고 본질적으로 변하지 않고 유지되어 왔습니다. 1960년대에는 Letraset 시트와 데스크탑 출판 소프트웨어의 출현으로 더욱 대중화되었습니다.
캐싱 된 토큰: 0
Prompt tokens: 137


In [26]:
with get_openai_callback() as cb:
    answer = llm.invoke(some_prompt.format('2문장으로 요약해봐'))
    chached_tokens = answer.response_metadata['token_usage']['prompt_tokens_details']['cached_tokens']
    print(cb)
    print(answer.content)
    print(f'캐싱 된 토큰: {chached_tokens}')
    print(f'Prompt tokens: {cb.prompt_tokens}')

Tokens Used: 217
	Prompt Tokens: 137
		Prompt Tokens Cached: 0
	Completion Tokens: 80
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $6.855e-05
Lorem Ipsum는 1500년대부터 인쇄 및 조판 산업에서 사용된 더미 텍스트로, 전자 조판으로의 전환에도 불구하고 본질적으로 변하지 않고 유지되어 왔습니다. 1960년대에는 Letraset 시트와 데스크탑 출판 소프트웨어의 출현으로 더욱 대중화되었습니다.
캐싱 된 토큰: 0
Prompt tokens: 137


In [27]:
with get_openai_callback() as cb:
    answer = llm.invoke(some_prompt.format('2문장으로 요약해봐'))
    chached_tokens = answer.response_metadata['token_usage']['prompt_tokens_details']['cached_tokens']
    print(cb)
    print(answer.content)
    print(f'캐싱 된 토큰: {chached_tokens}')
    print(f'Prompt tokens: {cb.prompt_tokens}')

Tokens Used: 215
	Prompt Tokens: 137
		Prompt Tokens Cached: 0
	Completion Tokens: 78
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $6.735e-05
Lorem Ipsum는 1500년대부터 인쇄 및 조판 산업에서 사용된 더미 텍스트로, 전자 조판으로의 전환에도 불구하고 본질적으로 변하지 않고 살아남았습니다. 1960년대에는 Letraset 시트와 데스크탑 출판 소프트웨어의 출현으로 더욱 대중화되었습니다.
캐싱 된 토큰: 0
Prompt tokens: 137


#### <b>모라게따 씨부랄꺼 뭐가 달러</b>

#### <b>이미지는 비싸니까 반복하는데다 사용하진 않겠음</b>